In [1]:
import torch
import torch.fft
import numpy as np

# --- 1. CONFIGURATION ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Simulating on: {DEVICE}")

# TRUE PARAMETERS (From your result)
# [log_phi1, log_phi2, log_phi3, log_phi4, advec_lat, advec_lon, log_nugget]
TRUE_LOG_PARAMS = torch.tensor([
    4.1875,   # log_phi1 
    1.9465,   # log_phi2 
    0.2492,   # log_phi3 
    -3.9739,  # log_phi4 
    0.0146,   # advec_lat
    -0.2040,  # advec_lon
    -0.8567   # log_nugget
], device=DEVICE, dtype=torch.float64)

# Mean Ozone
OZONE_MEAN = 260.0

# --- 2. EXACT COVARIANCE (Exponential / Matern v=0.5) ---
def get_model_covariance_on_grid(lags_x, lags_y, lags_t, params):
    phi1, phi2, phi3, phi4 = torch.exp(params[0]), torch.exp(params[1]), torch.exp(params[2]), torch.exp(params[3])
    advec_lat, advec_lon = params[4], params[5]
    sigmasq = phi1 / phi2

    # Grid is (Lat, Lon, Time)
    u_lat_eff = lags_x - advec_lat * lags_t
    u_lon_eff = lags_y - advec_lon * lags_t
    
    dist_sq = (u_lat_eff.pow(2) * phi3) + (u_lon_eff.pow(2)) + (lags_t.pow(2) * phi4)
    distance = torch.sqrt(dist_sq + 1e-8)
    
    return sigmasq * torch.exp(-distance * phi2)

# --- 3. FFT SIMULATION ---
def generate_exact_gems_field(lat_coords, lon_coords, t_steps, params):
    Nx = len(lat_coords)
    Ny = len(lon_coords)
    Nt = t_steps
    
    print(f"Exact Grid Size: {Nx} (Lat) x {Ny} (Lon) x {Nt} (Time) = {Nx*Ny*Nt} points")
    
    # Grid Steps
    dlat = float(lat_coords[1] - lat_coords[0])
    dlon = float(lon_coords[1] - lon_coords[0])
    
    # Padding
    Px, Py, Pt = 2*Nx, 2*Ny, 2*Nt
    
    # Lags Construction
    Lx_len = (lat_coords[-1] - lat_coords[0]) * (Px/Nx)
    lags_x = torch.arange(Px, device=DEVICE, dtype=torch.float64) * dlat
    lags_x[Px//2:] -= Lx_len 
    
    Ly_len = (lon_coords[-1] - lon_coords[0]) * (Py/Ny)
    lags_y = torch.arange(Py, device=DEVICE, dtype=torch.float64) * dlon
    lags_y[Py//2:] -= Ly_len

    dt = 1.0 # 1 Hour step
    Lt_len = Nt * dt * (Pt/Nt)
    lags_t = torch.arange(Pt, device=DEVICE, dtype=torch.float64) * dt
    lags_t[Pt//2:] -= Lt_len

    # Meshgrid & Covariance
    L_x, L_y, L_t = torch.meshgrid(lags_x, lags_y, lags_t, indexing='ij')
    C_vals = get_model_covariance_on_grid(L_x, L_y, L_t, params)

    # FFT & Convolution
    S = torch.fft.fftn(C_vals)
    S.real = torch.clamp(S.real, min=0)

    random_phase = torch.fft.fftn(torch.randn(Px, Py, Pt, device=DEVICE, dtype=torch.float64))
    weighted_freq = torch.sqrt(S.real) * random_phase
    field_sim = torch.fft.ifftn(weighted_freq).real
    
    # Crop to Exact Grid size
    return field_sim[:Nx, :Ny, :Nt]

# --- 4. EXECUTION ---
if __name__ == "__main__":
    
    # A. Define Exact Coordinates (Ascending for simulation)
    lats_sim = torch.arange(0, 5.0 + 0.001, 0.044, device=DEVICE, dtype=torch.float64)
    lons_sim = torch.arange(123.0, 133.0 + 0.001, 0.063, device=DEVICE, dtype=torch.float64)
    t_def = 8
    
    print("1. Generating True Field via FFT...")
    sim_field = generate_exact_gems_field(lats_sim, lons_sim, t_def, TRUE_LOG_PARAMS)
    
    # B. Formatting Output
    print("2. Formatting to Exact Regular Grid Tensors...")
    
    input_map = {}
    aggregated_list = [] # List to store tensors for concatenation
    
    nugget_std = torch.sqrt(torch.exp(TRUE_LOG_PARAMS[6]))
    
    # Flip for Descending Output order (Lat Max->0, Lon Max->123)
    lats_flip = torch.flip(lats_sim, dims=[0])
    lons_flip = torch.flip(lons_sim, dims=[0])
    
    # Meshgrid for Output
    grid_lat, grid_lon = torch.meshgrid(lats_flip, lons_flip, indexing='ij')
    flat_lats = grid_lat.flatten()
    flat_lons = grid_lon.flatten()
    
    for t in range(t_def):
        # Flip simulation field to match Descending coordinates
        field_t = sim_field[:, :, t] # [Lat_Asc, Lon_Asc]
        field_t_flipped = torch.flip(field_t, dims=[0, 1]) # [Lat_Desc, Lon_Desc]
        
        flat_vals = field_t_flipped.flatten()
        
        # Add Noise + Mean
        obs_vals = flat_vals + (torch.randn_like(flat_vals) * nugget_std) + OZONE_MEAN
        
        # Time Column (Start 21.0)
        time_val = 21.0 + t
        flat_times = torch.full_like(flat_lats, time_val)
        
        # Stack [Lat, Lon, Val, Time]
        row_tensor = torch.stack([flat_lats, flat_lons, obs_vals, flat_times], dim=1)
        
        # Generate Key
        key_str = f'2024_07_y24m07day01_hm{t:02d}:53'
        
        # Store in Input Map (CPU)
        input_map[key_str] = row_tensor.cpu()
        
        # Store for Aggregated Data (CPU)
        aggregated_list.append(row_tensor.cpu())

    # Create Single Big Tensor
    aggregated_data = torch.cat(aggregated_list, dim=0)

    # C. Verification
    first_key = list(input_map.keys())[0]
    tensor_ex = input_map[first_key]
    
    print(f"\nDone.")
    print(f"Aggregated Tensor Shape: {aggregated_data.shape}")
    print(f"Dictionary Size: {len(input_map)} keys.")
    print(f"Sample Top Rows from '{first_key}':")
    print(tensor_ex[:5])

Simulating on: cpu
1. Generating True Field via FFT...
Exact Grid Size: 114 (Lat) x 159 (Lon) x 8 (Time) = 145008 points
2. Formatting to Exact Regular Grid Tensors...

Done.
Aggregated Tensor Shape: torch.Size([145008, 4])
Dictionary Size: 8 keys.
Sample Top Rows from '2024_07_y24m07day01_hm00:53':
tensor([[  4.9720, 132.9540, 258.5280,  21.0000],
        [  4.9720, 132.8910, 258.6161,  21.0000],
        [  4.9720, 132.8280, 257.9525,  21.0000],
        [  4.9720, 132.7650, 261.4519,  21.0000],
        [  4.9720, 132.7020, 255.2433,  21.0000]], dtype=torch.float64)


In [2]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)
import os
import logging
import argparse # Argument parsing

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle
import torch
import torch.optim as optim
import copy                    # clone tensor
import time

# Custom imports


from GEMS_TCO import kernels_reparam_space_time_gpu as kernels_reparam_space_time
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import alg_optimization, alg_opt_Encoder

from typing import Optional, List, Tuple
from pathlib import Path
import typer
import json
from json import JSONEncoder
from GEMS_TCO import configuration as config
from GEMS_TCO.data_loader import load_data2, exact_location_filter
from GEMS_TCO import debiased_whittle
from torch.nn import Parameter

In [9]:
from GEMS_TCO import orderings as _orderings

def get_spatial_ordering(input_maps, mm_cond_number):
    # Extract the first timestamp's data
    key_list = list(input_maps.keys())
    data_for_coord = input_maps[key_list[0]]
    
    # --- FIX START ---
    # Check if input is Tensor, if so convert to Numpy for KDTree processing
    if isinstance(data_for_coord, torch.Tensor):
        data_for_coord = data_for_coord.cpu().numpy()
    # --- FIX END ---

    x1 = data_for_coord[:, 0]
    y1 = data_for_coord[:, 1]
    
    # Now this works because x1, y1 are numpy arrays
    coords1 = np.stack((x1, y1), axis=-1)
    
    # ... rest of your code ...
    # ord_mm = ...
    # nns_map = _orderings.find_nns_l2(locs=coords1, max_nn=mm_cond_number)
    # return ord_mm, nns_map


def get_spatial_ordering(
        
        input_maps: torch.Tensor,
        mm_cond_number: int = 10
    ) -> Tuple[np.ndarray, np.ndarray]:
        
        key_list = list(input_maps.keys())
        data_for_coord = input_maps[key_list[0]]
        
        # --- FIX START ---
        # Check if input is Tensor, if so convert to Numpy for KDTree processing
        if isinstance(data_for_coord, torch.Tensor):
            data_for_coord = data_for_coord.cpu().numpy()
        # --- FIX END ---

        x1 = data_for_coord[:, 0]
        y1 = data_for_coord[:, 1]
        
        # Now this works because x1, y1 are numpy arrays
        coords1 = np.stack((x1, y1), axis=-1)

        # Calculate MaxMin ordering
        ord_mm = _orderings.maxmin_cpp(coords1)
        
        # Reorder coordinates to find nearest neighbors
        data_for_coord_reordered = data_for_coord[ord_mm]
        coords1_reordered = np.stack(
            (data_for_coord_reordered[:, 0], data_for_coord_reordered[:, 1]), 
            axis=-1
        )
        
        # Calculate nearest neighbors map
        nns_map = _orderings.find_nns_l2(locs=coords1_reordered, max_nn=mm_cond_number)
        
        return ord_mm, nns_map

ord_mm, nns_map = get_spatial_ordering(input_map, mm_cond_number=10)

In [15]:
mm_input_map = {}
for key in input_map:
    mm_input_map[key] = input_map[key][ord_mm]  # Extract only Lat and Lon columns

# Fit vecchia max min

In [ ]:
v=0.5
mm_cond_number= 8
nheads = 0
lr = 0.1
patience, factor = 5, 0.5


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# Global L-BFGS Settings
LBFGS_LR = 1.0
LBFGS_MAX_STEPS = 10      # 10 to 20  
LBFGS_HISTORY_SIZE = 100   
LBFGS_MAX_EVAL = 100       # line search from 50 to 80
       

DELTA_LAT, DELTA_LON = 0.044, 0.063 
LAT_COL, LON_COL, VAL_COL, TIME_COL = 0, 1, 2, 3

days_list = [1]

# --- 2. Run optimization loop over pre-loaded data ---

for day_idx in days_list:  # 0-based

    print(f'\n{"="*40}')
    print(f'--- Starting Processing for Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'{"="*40}')



    if isinstance(aggregated_data, torch.Tensor):
        aggregated_data = aggregated_data.to(DEVICE)


    init_sigmasq   = 13.059
    init_range_lat = 0.154 
    init_range_lon = 0.195
    init_advec_lat = 0.0218
    init_range_time = 1.0
    init_advec_lon = -0.1689
    init_nugget    = 0.247
    
    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                # 1/range_lon
    init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
    init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
    init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

    # Create Initial Parameters (Float64, Requires Grad)
    initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                    np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]


    params_list = [
        torch.tensor([val], requires_grad=True, dtype=torch.float64, device=DEVICE)
        for val in initial_vals
    ]


    # --- 💥 Instantiate the GPU Batched Class ---
    # NOTE: Ensure fit_vecchia_lbfgs is the NEW class we defined
    model_instance = kernels_reparam_space_time.fit_vecchia_lbfgs(
            smooth = v,
            input_map = mm_input_map,
            aggregated_data = aggregated_data,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )


    # --- 💥 Set L-BFGS Optimizer ---
    optimizer_vecc = model_instance.set_optimizer(
                params_list,     
                lr=LBFGS_LR,            
                max_iter=LBFGS_MAX_EVAL,        
                history_size=LBFGS_HISTORY_SIZE 
            )

    print(f"\n--- Starting Phase 2: Vecchia Optimization (Day {day_idx+1}) ---")
    start_time = time.time()
    
    # --- 💥 Call the Batched Fit Method ---
    # REMOVED: model_instance.matern_cov_aniso_STABLE_log_reparam
    out, steps_ran = model_instance.fit_vecc_lbfgs(
            params_list,
            optimizer_vecc,
            # covariance_function argument is GONE
            max_steps=LBFGS_MAX_STEPS, 
            grad_tol=1e-7
        )

    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Vecchia Optimization finished in {epoch_time:.2f}s. Results: {out}")

Using device: cpu

--- Starting Processing for Day 2 (2024-07-2) ---

--- Starting Phase 2: Vecchia Optimization (Day 2) ---
Pre-computing Batched Tensors (Padding Strategy)... Done. Heads: 0, Batched Tails: 145008
--- Starting Batched L-BFGS Optimization (GPU) ---


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn